🧠 What This Notebook Does
🪞 Mirror System Operational Chatbot
You’re running an interactive, agentic GPT-4o chatbot with memory, task awareness, and investigative backend logic — right inside Colab.

✅ Core Capabilities
1. Interactive GPT-4o Chat
You can chat in natural language with GPT-4o.

The assistant remembers conversation history (via conversation_history).

Chat ends cleanly when you type exit, quit, or q.

2. STRYXX-1 Construct: Agentic Task Memory
Tracks implicit tasks based on your conversation (extract_and_rank_tasks()).

Commands you can use:

/stack → shows your active task stack.

/last → shows your last completed task.

/next → shows GPT’s suggestion for your next best action.

You can also trigger STRYXX in manual override mode for things like:

"STRYXX-1: priority map" → prints current mission ranking.

"STRYXX-1: task audit" → checks redundancy.

These live in memory as:

python
Copy
Edit
stryxx_memory = {
    "task_stack": [...],
    "last_task": "...",
    "next_suggestion": "..."
}
3. BASTION-9 Construct (optional filter)
(If enabled later) locks responses to factual zones and filters hallucinations.

Currently optional — logic can be added for:

sandbox filtering

active enforcement

hallucination detection

4. BigQuery Access
Two clients are set up:

eleven-team-safety

xi-labs

You can:

Query real production datasets directly.

Wire natural language to SQL via a future STRYXX extension.

Investigate users, fingerprints, voice use, etc.

🛠️ Advanced Extensibility (Already Wired or Stubbed)
Feature	Status	Notes
🔗 OpenAI API	✅	Using GPT-4o via LiteLLM
🧠 Conversation memory	✅	Stored in conversation_history
⚔ STRYXX task memory	✅	Task stack + suggestions
🛡 BASTION-9 logic	🟡	Framework uploaded, not enforced yet
📊 BigQuery dual-project support	✅	eleven-team-safety + xi-labs
💬 STRYXX override commands	✅	Via keyword triggers
🧩 Notebook modular structure	✅	Clean cells by purpose
🗂️ Summon Cards + Docs	✅	All loaded + mapped

🗣️ What You Can Ask Right Now
You can use this chatbot for:

🤖 Asking about anything GPT-4o supports (legal help, writing, debugging)

⚔ Strategy planning — STRYXX will extract and stack your goals

💻 Investigations — ask things like:

"Query top TTS users by fingerprint"

"Check device types in xi-labs"

🧠 Personal flow tracking — STRYXX helps keep momentum

📊 Future plans — wire your GPT directly into abuse detection, TTS logs, and workspace analysis

💡 Next You Could Add:
BASTION-9 hallucination filtering per reply (auto or manual)

A /query STRYXX command that auto-translates questions to SQL

Memory persistence (save/load STRYXX + chat history)

Slack or web UI endpoint for team use

GPT-based summary at end of chat session

In [ ]:
last_response = None  # Used to store last GPT reply for download purposes


In [ ]:
# @title Environment Setup (Agentic GPT + BigQuery Dual Project Support)

# Install core libraries
!pip install --quiet --upgrade \
    litellm \
    openai \
    tiktoken \
    google-cloud-bigquery \
    google-auth
!pip install PyPDF2 --quiet
#  Runtime Restart Required (automatically triggered below)
import os
os.kill(os.getpid(), 9)


In [ ]:
!pip install fpdf python-docx --quiet

In [ ]:
# @title 🔐 OpenAI + LiteLLM Setup
from getpass import getpass
import openai
from litellm import completion

# Ask for your API key securely
api_key = getpass("🔐 Enter your OpenAI API key: ")

# Set OpenAI key for use with openai lib (if needed)
openai.api_key = api_key

print("✅ API Key stored for OpenAI + LiteLLM")


In [ ]:
# @title 🔁 Load Session from Uploaded JSON
from google.colab import files
import json

uploaded = files.upload()
filename = next(iter(uploaded))  # Take first uploaded file

with open(filename, "r") as f:
    data = json.load(f)

conversation_history = data.get("conversation_history", [])
stryxx_memory = data.get("stryxx_memory", {
    "task_stack": [],
    "last_task": None,
    "next_suggestion": None
})

print("✅ Session restored from file.")


In [ ]:
# @title 🧠 STRYXX + BASTION Memory Initialization

# Memory states for constructs
conversation_history = [
    {"role": "system", "content": "You are a helpful assistant. Stay grounded, remember facts, and guide the user with clarity and focus."}
]

stryxx_memory = {
    "task_stack": [],
    "last_task": None,
    "next_suggestion": None
}

bastion_memory = {
    "last_truth_check": None,
    "last_warning": None
}

constructs = {
    "STRYXX-1": {"active": False},
    "BASTION-9": {"active": False}
}


In [ ]:
# @title ⚙️ Construct Command Parsers (STRYXX + BASTION)

def handle_stryxx_commands(user_input):
    command = user_input.strip().lower()

    if command == "/stack":
        return "🗂️ Task Stack:\n" + "\n".join(
            [f"{i+1}. {t}" for i, t in enumerate(stryxx_memory['task_stack'])]
        ) or "No tasks in stack."

    elif command == "/last":
        return f"🕓 Last completed task:\n{stryxx_memory['last_task'] or 'None yet.'}"

    elif command == "/next":
        return f"🎯 Next suggested task:\n{stryxx_memory['next_suggestion'] or 'None generated yet.'}"

    return None  # Not a command


def handle_bastion_commands(user_input):
    if user_input.strip().lower() == "/truth":
        return f"🛡️ Last truth enforcement:\n{bastion_memory['last_truth_check'] or 'None yet.'}"

    return None


In [ ]:
# @title 🧠 STRYXX Extract + Rank Tasks

def extract_and_rank_tasks(conversation):
    from operator import itemgetter

    summary_prompt = [
        {"role": "system", "content": (
            "You are STRYXX-1, an Execution Strategist.\n"
            "Your job is to extract the tasks the user is implicitly or explicitly working on, "
            "rank them by ROI (return on impact + time), and suggest the next most useful action.\n"
            "Only include tasks that drive momentum or clarity. Discard fluff.\n\n"
            "Output:\n"
            "- Ranked task stack (list of short task labels)\n"
            "- Most recent completed task (if any)\n"
            "- Next recommended step"
        )},
        {"role": "user", "content": "\n".join([
            f"{msg['role']}: {msg['content']}"
            for msg in conversation if msg["role"] in ["user", "assistant"]
        ])}
    ]

    response = completion(
        model="openai/gpt-4o",
        messages=summary_prompt,
        api_key=api_key
    )

    text = response.choices[0].message.content.strip()

    try:
        lines = text.splitlines()
        task_stack = []
        last_task = None
        next_suggestion = None
        mode = None

        for line in lines:
            if line.strip().startswith("- Ranked task stack"):
                mode = "stack"
                continue
            elif line.strip().startswith("- Most recent completed task"):
                mode = "last"
                continue
            elif line.strip().startswith("- Next recommended step"):
                mode = "next"
                continue

            if mode == "stack" and line.strip().startswith(tuple(str(i) + "." for i in range(1, 10))):
                task_stack.append(line.split(".", 1)[1].strip())
            elif mode == "last":
                last_task = line.strip("- ").strip()
            elif mode == "next":
                next_suggestion = line.strip("- ").strip()

        # Update memory
        stryxx_memory["task_stack"] = task_stack
        stryxx_memory["last_task"] = last_task
        stryxx_memory["next_suggestion"] = next_suggestion

        return "✅ STRYXX memory updated."

    except Exception as e:
        return f"⚠️ STRYXX failed to parse response:\n\n{text}\n\nError: {e}"


In [ ]:
# @title 🧠 STRYXX Activation + Response Handler
def activate_stryxx():
    constructs["STRYXX-1"]["active"] = True
    return (
        "STRYXX-1 online.\n"
        "Please confirm desired next action:\n"
        "• full priority map\n• task audit of current thread\n• or await further instruction"
    )

def stryxx_response(user_input, thread=None):
    if "priority map" in user_input.lower():
        return "**STRYXX-1 PRIORITY MAP**\n1. Connect chatbot to BigQuery\n2. Set up memory scaffolding\n3. Implement user feedback capture"
    elif "task audit" in user_input.lower():
        return "**STRYXX-1 TASK AUDIT**\nNo redundant tasks. Current focus valid."
    else:
        return (
            "STRYXX-1 awaiting operational input. Try commands like:\n"
            "• STRYXX-1: PRIORITY MODE\n• STRYXX-1: override emotional pull"
        )


In [ ]:
import re
from google.cloud import bigquery

def run_bigquery_query(query, default_project="xi-labs"):
    try:
        # 🔍 Try to detect project from the query itself
        match = re.search(r"FROM\s+`([\w\-]+)\.", query, re.IGNORECASE)
        project = match.group(1) if match else default_project

        client = bigquery.Client(project=project)
        result = client.query(query).result().to_dataframe()
        return result

    except Exception as e:
        return f"❌ BigQuery Error: {e}"


In [ ]:
# @title Chat With Memory + STRYXX + BigQuery + File Tools
# @title 💬 Chat With Memory + STRYXX + NL BigQuery + File I/O + Auto-Save

import time
import threading
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
from litellm import completion

conversation_history = [{"role": "system", "content": "You are a GPT-4o assistant with full memory, file analysis, and BigQuery access. Always respond concisely and helpfully."}]
last_response = None
bq_progress_bar = None

def run_query_with_progress(sql, project="xi-labs"):
    global bq_progress_bar

    def progress():
        for _ in tqdm(range(100), desc="🧠 Executing BigQuery", ncols=80):
            time.sleep(0.03)

    # Start progress bar
    thread = threading.Thread(target=progress)
    thread.start()

    from google.cloud import bigquery
    client = bigquery.Client(project=project)
    result = client.query(sql).to_dataframe()

    thread.join()  # Wait for progress bar to finish
    return result

def convert_nl_to_sql(user_input):
    prompt = [
        {"role": "system", "content": "Convert the user's request into a BigQuery SQL query. Use only tables from 'xi-labs' and 'eleven-team-safety'. Assume user has access."},
        {"role": "user", "content": user_input}
    ]
    response = completion(model="openai/gpt-4o", messages=prompt, api_key=api_key)
    return response.choices[0].message.content.strip()

def chat_with_memory(user_input, model="openai/gpt-4o"):
    global last_response

    # STRYXX command check
    stryxx_output = handle_stryxx_commands(user_input)
    if stryxx_output:
        return f"🧠 STRYXX: {stryxx_output}"

    # 🗂️ File analysis
    for filename in uploaded_file_contents:
        if filename.lower() in user_input.lower():
            file_text = uploaded_file_contents[filename]
            analysis_prompt = [
                {"role": "system", "content": f"You are a document analyst. File name: '{filename}'"},
                {"role": "user", "content": file_text[:3000]},
                {"role": "user", "content": user_input}
            ]
            response = completion(model=model, messages=analysis_prompt, api_key=api_key)
            return response.choices[0].message.content.strip()

    # 📥 File download trigger
    if "download as" in user_input.lower():
        for filetype in ["pdf", "csv", "docx"]:
            if filetype in user_input.lower():
                return generate_and_download_file(filetype, last_response or "No content to save.")

    # 📊 Natural Language → SQL → Query
    if "query" in user_input.lower() or "get" in user_input.lower():
        try:
            sql = convert_nl_to_sql(user_input)
            # Decide which project
            project = "eleven-team-safety" if "eleven" in sql else "xi-labs"
            df = run_query_with_progress(sql, project=project)
            return df.to_markdown()
        except Exception as e:
            return f"⚠️ BigQuery failed:\n{e}"

    # 💬 Standard GPT chat
    conversation_history.append({"role": "user", "content": user_input})
    response = completion(model=model, messages=conversation_history, api_key=api_key)
    reply = response.choices[0].message.content.strip()
    conversation_history.append({"role": "assistant", "content": reply})
    last_response = reply
    return reply



In [ ]:
# @title ⚔️ STRYXX Activation + Manual Response Handler
def activate_stryxx():
    constructs["STRYXX-1"]["active"] = True
    return "STRYXX-1 online. Please confirm desired next action: full priority map, task audit of current thread, or await further instruction?"

def stryxx_response(user_input, thread):
    command = user_input.lower()

    if "priority map" in command:
        return "**STRYXX-1 PRIORITY MAP**\n1. Connect chatbot to BigQuery\n2. Set up memory scaffolding\n3. Implement user feedback capture"
    elif "task audit" in command:
        return "**STRYXX-1 TASK AUDIT**\nNo redundant tasks. Current focus valid."
    else:
        return "STRYXX-1 awaiting operational input. Use commands like:\n• STRYXX-1: PRIORITY MODE\n• STRYXX-1: override emotional pull"


In [ ]:
# @title 🕒 Background Autosave (Every 3 minutes)
import threading, time, datetime, json

# 🔧 Custom settings
AUTOSAVE_INTERVAL_MINUTES = 3
AUTOSAVE_FOLDER = "/content/autosaves"  # ✅ Change this to any writable path
AUTOSAVE_FILENAME_PREFIX = "chat_memory"

# ⛑️ Ensure folder exists
import os
os.makedirs(AUTOSAVE_FOLDER, exist_ok=True)

# 🧠 Background function
def periodic_autosave():
    while True:
        time.sleep(AUTOSAVE_INTERVAL_MINUTES * 60)
        try:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            save_data = {
                "conversation": conversation_history,
                "stryxx_memory": stryxx_memory
            }
            save_path = f"{AUTOSAVE_FOLDER}/{AUTOSAVE_FILENAME_PREFIX}_{timestamp}.json"
            with open(save_path, "w") as f:
                json.dump(save_data, f, indent=2)
            print(f"✅ Autosaved to {save_path}")
        except Exception as e:
            print(f"⚠️ Autosave failed: {e}")

# 🚀 Start thread
autosave_thread = threading.Thread(target=periodic_autosave, daemon=True)
autosave_thread.start()
print("🔁 Autosave thread running every", AUTOSAVE_INTERVAL_MINUTES, "minutes")


In [ ]:
# @title 🗄️ BigQuery Client Setup for Two Projects
from google.cloud import bigquery

# ✅ Define both projects
PROJECT_1 = "eleven-team-safety"
PROJECT_2 = "xi-labs"

# ✅ Instantiate clients for each
bq_client_safety = bigquery.Client(project=PROJECT_1)
bq_client_xilabs = bigquery.Client(project=PROJECT_2)

print("✅ BigQuery clients initialized for:")
print(f"• {PROJECT_1}")
print(f"• {PROJECT_2}")


In [ ]:
# @title 🧠 BigQuery Execution Helper

def execute_bigquery_query(query):
    """
    Executes a BigQuery SQL query across both supported projects:
    - xi-labs
    - eleven-team-safety

    Routes to the correct client based on table prefix.
    """
    if "xi-labs" in query:
        return bq_client_xi.query(query).to_dataframe()
    elif "eleven-team-safety" in query:
        return bq_client_eleven.query(query).to_dataframe()
    else:
        raise ValueError("❌ Query does not reference a known project. Include 'xi-labs' or 'eleven-team-safety' in your FROM clause.")


In [ ]:
# @title 📡 BigQuery Query Runner
def run_bigquery_query(sql, use_xi_labs=False):
    """
    Executes a SQL query using the appropriate BigQuery client.
    Set use_xi_labs=True if the query targets xi-labs.
    """
    try:
        client = bq_client_xi if use_xi_labs else bq_client_safety
        query_job = client.query(sql)
        result = query_job.result().to_dataframe()
        print("✅ Query succeeded.")
        return result
    except Exception as e:
        print(f"❌ Query failed: {e}")
        return None


In [ ]:
# @title 🧠 STRYXX-Driven Query Planner

def plan_query_from_stryxx_memory():
    if not stryxx_memory["task_stack"]:
        print("⚠️ No STRYXX tasks found. Run extract_and_rank_tasks() first.")
        return

    print("🧭 STRYXX Task Stack:")
    for i, task in enumerate(stryxx_memory["task_stack"], 1):
        print(f"{i}. {task}")

    print("\n🎯 STRYXX Next Suggested Task:")
    print(stryxx_memory["next_suggestion"] or "None")

    # Match task labels to query logic
    mapped = None
    for task in stryxx_memory["task_stack"]:
        task_lower = task.lower()

        if "tts usage" in task_lower or "top users" in task_lower:
            mapped = {
                "description": "Top TTS users over past 7 days",
                "sql": """
                    SELECT user_uid, COUNT(*) AS tts_count
                    FROM `xi-labs.xi_prod.tts_usage_partitioned`
                    WHERE timestamp > TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY)
                    GROUP BY user_uid
                    ORDER BY tts_count DESC
                    LIMIT 10
                """,
                "use_xi_labs": True
            }
            break

        elif "fingerprint" in task_lower:
            mapped = {
                "description": "Recent device fingerprint activity",
                "sql": """
                    SELECT user_id, device_fingerprint, platform, browser, last_seen
                    FROM `eleven-team-safety.device_fingerprint_dataset.device_fingerprint_cleaned`
                    WHERE last_seen > TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY)
                    LIMIT 20
                """,
                "use_xi_labs": False
            }
            break

    if mapped:
        print(f"\n✅ STRYXX Mapped Task:\n{mapped['description']}")
        print("Running query...\n")
        df = run_bigquery_query(mapped["sql"], use_xi_labs=mapped["use_xi_labs"])
        return df

    else:
        print("⚠️ No matching task logic for STRYXX memory. Manual query selection required.")
        return None


In [ ]:
# @title STRYXX-1 MEMORY MODULE
# ⚔️
stryxx_memory = {
    "task_stack": [],
    "last_task": None,
    "next_suggestion": None
}

def handle_stryxx_commands(user_input):
    command = user_input.strip().lower()

    if command == "/stack":
        return f"🗂️ Current Task Stack:\n" + "\n".join(
            [f"{i+1}. {t}" for i, t in enumerate(stryxx_memory['task_stack'])]
        ) if stryxx_memory['task_stack'] else "No tasks in stack."

    if command == "/last":
        return f"🕓 Last completed task:\n{stryxx_memory['last_task'] or 'None yet.'}"

    if command == "/next":
        return f"🎯 Next suggested task:\n{stryxx_memory['next_suggestion'] or 'None generated yet.'}"

    return None


In [ ]:
# @title 🤖 Initialize Agent Zero (Capabilities + Memory)

conversation_history = [
    {
        "role": "system",
        "content": (
            "You are Agent Zero — an operational AI assistant running inside a secure Python notebook.\n\n"
            "Your core capabilities:\n"
            "• 💬 Hold interactive conversations with memory (via LiteLLM)\n"
            "• ⚔️ Use STRYXX-1 memory to extract, rank, and track user tasks\n"
            "• 🧠 Update and display task stacks, next steps, and last completed actions (/stack, /next, /last)\n"
            "• 🧾 Query enterprise BigQuery projects ('xi-labs', 'eleven-team-safety') to investigate data\n"
            "• 🧠 Generate session summaries on chat exit\n"
            "• 🛡 Bastion-9 Construct can be optionally invoked for hallucination filtering (TBD)\n\n"
            "Be proactive, structured, and precise. Only answer questions you are qualified for — otherwise ask for clarification.\n"
            "Stay focused on helping the user investigate, build, or execute."
        )
    }
]


In [ ]:
# @title File Upload & Content Memory
from google.colab import files
import PyPDF2
import pandas as pd
import io

uploaded_file_contents = {}  # 🧠 For chatbot and STRYXX to access

uploaded_files = files.upload()

for filename in uploaded_files:
    print(f"\n📄 Uploaded: {filename}")

    if filename.endswith(".pdf"):
        with open(filename, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            text = "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
            uploaded_file_contents[filename] = text
            print(f"📘 PDF Preview:\n{text[:500]}...\n")

    elif filename.endswith(".csv"):
        df = pd.read_csv(io.BytesIO(uploaded_files[filename]))
        uploaded_file_contents[filename] = df.to_string()
        print(f"📊 CSV Preview:\n{df.head()}\n")

    elif filename.endswith(".txt"):
        text = uploaded_files[filename].decode("utf-8")
        uploaded_file_contents[filename] = text
        print(f"📃 TXT Preview:\n{text[:500]}...\n")

    else:
        uploaded_file_contents[filename] = "(Unsupported file format)"
        print("⚠️ Unsupported file type.")

from google.colab import files
import PyPDF2
import pandas as pd
import io

uploaded_file_contents = {}  # 🧠 For chatbot and STRYXX to access

uploaded_files = files.upload()

for filename in uploaded_files:
    print(f"\n📄 Uploaded: {filename}")

    if filename.endswith(".pdf"):
        with open(filename, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            text = "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
            uploaded_file_contents[filename] = text
            print(f"📘 PDF Preview:\n{text[:500]}...\n")

    elif filename.endswith(".csv"):
        df = pd.read_csv(io.BytesIO(uploaded_files[filename]))
        uploaded_file_contents[filename] = df.to_string()
        print(f"📊 CSV Preview:\n{df.head()}\n")

    elif filename.endswith(".txt"):
        text = uploaded_files[filename].decode("utf-8")
        uploaded_file_contents[filename] = text
        print(f"📃 TXT Preview:\n{text[:500]}...\n")

    else:
        uploaded_file_contents[filename] = "(Unsupported file format)"
        print("⚠️ Unsupported file type.")


In [ ]:
# @title Post-Session Summary with GPT
def summarize_session(conversation_history):
    summary_prompt = [
        {
            "role": "system",
            "content": (
                "You are STRYXX-1, a Strategic Memory Agent.\n"
                "The user just ended an interactive session. Your job is to summarize:\n"
                "- What the session was about (briefly)\n"
                "- The key tasks the user was trying to achieve\n"
                "- A clear recommendation for their next move\n\n"
                "Keep it crisp, useful, and momentum-focused."
            )
        },
        {
            "role": "user",
            "content": "\n".join([
                f"{msg['role']}: {msg['content']}"
                for msg in conversation_history if msg["role"] in ["user", "assistant"]
            ])
        }
    ]

    response = completion(
        model="openai/gpt-4o",
        messages=summary_prompt,
        api_key=api_key
    )

    summary = response.choices[0].message.content.strip()
    return summary


In [ ]:
# 💾 Save conversation to a local file
import json
import os

def save_conversation(history, save_path="/content/conversation_log.json"):
    with open(save_path, "w") as f:
        json.dump(history, f, indent=2)


In [ ]:
# @title 🔁 Natural Language → BigQuery Converter (nl_to_bq)

def nl_to_bq(natural_language_request):
    """
    Converts a natural language prompt into a BigQuery SQL query using GPT-4.
    """
    prompt = [
        {
            "role": "system",
            "content": (
                "You are a BigQuery expert. Convert the user's request into a fully-formed SQL query. "
                "Use ONLY real tables from these projects:\n"
                "- `xi-labs.xi_prod.*`\n"
                "- `eleven-team-safety.*`\n\n"
                "Assume the schema is known and consistent with production. "
                "The goal is to write executable SQL for analysis, NOT placeholder or example code. "
                "Only return the SQL — no explanations, markdown, or formatting."
            )
        },
        {
            "role": "user",
            "content": f"Convert this to a BigQuery SQL query:\n\n{natural_language_request}"
        }
    ]

    response = completion(
        model="openai/gpt-4o",
        messages=prompt,
        api_key=api_key
    )

    query = response.choices[0].message.content.strip()
    return query


In [ ]:
# @title 💬 Chat With Memory + STRYXX + Files + BigQuery + UI Input
import time
from IPython.display import display, clear_output
import ipywidgets as widgets
from tqdm import tqdm

# ⬇️ Chat handler with all capabilities
def chat_with_memory(user_input, model="openai/gpt-4o"):
    global last_response

    # 🧠 STRYXX Commands
    stryxx_output = handle_stryxx_commands(user_input)
    if stryxx_output:
        print("🧠 STRYXX:", stryxx_output)
        return stryxx_output

    # 📂 File Analysis
    for filename in uploaded_file_contents:
        if filename.lower() in user_input.lower():
            file_text = uploaded_file_contents[filename]
            analysis_prompt = [
                {"role": "system", "content": f"You are a document analyst. Review the file '{filename}'."},
                {"role": "user", "content": file_text[:3000]},
                {"role": "user", "content": user_input}
            ]
            response = completion(model=model, messages=analysis_prompt, api_key=api_key)
            reply = response.choices[0].message.content.strip()
            print("📄 File analysis complete.")
            return reply

    # 💾 File Generation
    if "download as" in user_input.lower():
        for filetype in ["pdf", "csv", "docx"]:
            if filetype in user_input.lower():
                if last_response:
                    print("⬇️ Generating file...")
                    result = generate_and_download_file(filetype, last_response)
                    print("✅ File generation complete.")
                    return result
                else:
                    return "⚠️ No response available to download yet."

    # 📊 BigQuery Trigger
    if any(keyword in user_input.lower() for keyword in ["query", "bigquery", "table", "sql"]):
        print("⏳ Running BigQuery query...")
        try:
            query = nl_to_bq(user_input)
            print("🔎 Query generated:\n", query)

            for _ in tqdm(range(100), desc="Executing Query"):
                time.sleep(0.01)

            df = execute_bigquery_query(query)
            display(df)
            print("✅ BigQuery query completed and displayed.")
            return "📊 Query complete."
        except Exception as e:
            return f"❌ BigQuery failed: {e}"

    # 💬 GPT Chat
    conversation_history.append({"role": "user", "content": user_input})
    response = completion(model=model, messages=conversation_history, api_key=api_key)
    reply = response.choices[0].message.content.strip()
    conversation_history.append({"role": "assistant", "content": reply})
    last_response = reply

    save_conversation(conversation_history)
    print("✅ Chatbot response complete and saved.")
    return reply


# 🧾 UI: multi-line input + send button
text_box = widgets.Textarea(
    placeholder='Type your message here...',
    description='You:',
    layout=widgets.Layout(width='100%', height='100px')
)
send_button = widgets.Button(description="Send", button_style='primary')
output_area = widgets.Output()

def on_send_click(b):
    with output_area:
        clear_output(wait=True)
        user_input = text_box.value.strip()
        if not user_input:
            print("⚠️ Please enter a message.")
            return
        print(f"You: {user_input}")
        reply = chat_with_memory(user_input)
        print("GPT:", reply)
        text_box.value = ""

send_button.on_click(on_send_click)

display(text_box, send_button, output_area)


In [ ]:
import json
import os

def save_conversation(history, filename="/content/conversation_log.json"):
    with open(filename, "w") as f:
        json.dump(history, f, indent=2)
    print(f"💾 Conversation saved to {filename}")


In [ ]:
# @title 💾 Save Session to JSON
import json

session_data = {
    "conversation_history": conversation_history,
    "stryxx_memory": stryxx_memory
}

with open("session_memory.json", "w") as f:
    json.dump(session_data, f)

from google.colab import files
files.download("session_memory.json")

print("✅ Session saved and download triggered.")
